In [97]:
import pandas as pd
import numpy as np

import scipy.stats
from statsmodels.stats.weightstats import *
from statsmodels.stats.descriptivestats import sign_test

In [102]:
leicoz_data = np.array([49,58,75,110,112,132,151,276,281,362])

In [103]:
stats.wilcoxon(leicoz_data - 200)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [13]:
deforestation = [22,22,15,13,19,19,18,20,21,13,13,15]
no_deforestation = [17,18,18,15,12,4,14,15,10]

In [15]:
stats.mannwhitneyu(deforestation, no_deforestation)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [18]:
challenger_data = pd.read_csv('challenger.txt', sep='\t', header=0)
challenger_data.columns = ['Date', 'Temperature', 'Incident']

In [21]:
challenger_data.drop(['Date'], axis=1, inplace=True)

In [23]:
challenger_data.head()

,Temperature,Incident
0,18.9,0
1,21.1,1
2,20.6,0
3,20.0,0
4,19.4,0


In [53]:
seal_damage = challenger_data[challenger_data['Incident'] == 1].drop(['Incident'], axis=1)
no_seal_damage = challenger_data[challenger_data['Incident'] == 0].drop(['Incident'], axis=1)

In [78]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [79]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [85]:
np.random.seed(0)

seal_damage_score = list(map(np.mean, get_bootstrap_samples(seal_damage, 1000)))
no_seal_damage_score = list(map(np.mean, get_bootstrap_samples(no_seal_damage, 1000)))

print ("95% confidence interval for the seal_damage:",  stat_intervals(seal_damage_score, 0.05))
print ("95% confidence interval for the no_seal_damage:",  stat_intervals(no_seal_damage_score, 0.05))

95% confidence interval for the seal_damage: [14.61428571 20.71607143]
95% confidence interval for the no_seal_damage: [21.14359375 23.55046875]


In [86]:
delta_mean_scores = list(map(lambda x: x[1] - x[0], zip(seal_damage_score, no_seal_damage_score)))

In [87]:
print ("95% confidence interval for the difference between mean",  stat_intervals(delta_mean_scores, 0.05))

95% confidence interval for the difference between mean [1.45040179 8.06457589]


In [88]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [89]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [90]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [91]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [94]:
np.random.seed(0)
print ("p-value: %f" % permutation_test(seal_damage, no_seal_damage, max_permutations = 10000))

p-value: 0.005700
